In [30]:
import pandas as pd
df = pd.read_csv('last_n_of_samples.csv')
df

,Tweet,Location,date,hours,minuites
0,اتصالات شركه نصابه وبتضحك ع العملاء ومفيش اي ...,"Cairo, Egypt",2022-05-26,12,00:00:00
1,ممكن التواصل معي الخاص أجل تقديم شكوى ف شركة ...,"Cairo, Egypt",2022-05-23,20,00:00:00
2,ممكن التواصل الخاص تكرمتم,"Cairo, Egypt",2022-05-23,19,00:00:00
3,عايز الايميل الرسمي بتاعكم انتوا الإمارات ينف...,"Cairo, Egypt",2022-05-23,19,00:00:00
4,اتصالات مصر معندهاش ايميل لتلقي الشكاوي وبتقو...,"Cairo, Egypt",2022-05-23,19,00:00:00
...,...,...,...,...,...
304,ممكن تغيير بيانات الخط اقرب فرع يكون متواجد ح...,"Cairo, Egypt",2022-05-22,6,00:00:00
305,السلام ورحمة الله وبركاته انا سافرت مصر وشريح...,"Cairo, Egypt",2022-05-21,14,00:00:00
306,الشريحه متوقفه واريد تفعيلها مصالحنا متعطله و...,"Cairo, Egypt",2022-05-21,10,00:00:00
307,عندك خاصية بالجوال حقك القائمة البيضاء فعلها ...,"القاهرة, مصر",2022-05-21,1,00:00:00


In [22]:
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.sentiment import SentimentAnalyzer
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger
from functools import lru_cache


    
class Morphological_Analysis():

    def __init__(self, DF, COL):
        self.df = DF.copy()
        self.col = COL
        self.mled = MLEDisambiguator.pretrained()
        self.tagger = DefaultTagger(self.mled, 'pos')
        self.mapping = {}
        

    def del_nonAR(self):
        for rows in self.df[self.col]:
            for char in rows:
                if char not in AR_LETTERS_CHARSET and char != ' ' :
                    self.df[self.col] = self.df[self.col].str.replace(char, '')
        return self.df

    '''                
    def get_lex(self):
        
        db = MorphologyDB.builtin_db('calima-egy-r13')
        analyzer = Analyzer(db)
        
        for rows in df[self.col]:
            sentence = simple_word_tokenize(rows)
            lex_sentence = ''
            for words in sentence:
                try:
                    lex_sentence += analyzer.analyze(words)[0]['root'] + ' '
                    lex_sentence = lex_sentence.replace('.', '')
                except:
                    lex_sentence += ' ' + words + ' '
            print(lex_sentence)
            self.df[self.col] = self.df[self.col].str.replace(rows, lex_sentence)
        return df
    '''
    
    def replace_rows(self, rows):
        Cleaned_Sent = ' '
        Split_Sent = rows.split()

        for words in Split_Sent:

            if words in self.mapping:
                Tags = self.mapping[words]
            else:
                Tags = self.tagger.tag(words.split())
                self.mapping[words] = Tags

            if Tags[0] != 'conj' and Tags[0] != 'prep':
                Cleaned_Sent = Cleaned_Sent + words + " "
        return Cleaned_Sent
    
    def del_stopwords(self):
        self.df[self.col] = self.df[self.col].apply(self.replace_rows)
        return self.df
            
class Sentiment_Analysis:
    
    def __init__(self, DF, COL):
        self.df = DF
        self.col = COL
    
    def get_sentiment(self):
        sa = SentimentAnalyzer.pretrained()
        sentences = []
        for rows in self.df[self.col]:
            sentences.append(rows)
        sentiments = sa.predict(sentences)
        self.df['Sentiment'] = sentiments
        return self.df

Removing Non-Arabic Characters

In [23]:
Morph = Morphological_Analysis(df, 'Tweet')


In [ ]:
df = Morph.del_nonAR()
df

In [24]:
df = df.drop_duplicates(subset = 'Tweet')
df

,Tweet,Location,date,hours,minuites
0,اتصالات شركه نصابه وبتضحك ع العملاء ومفيش اي ...,"Cairo, Egypt",2022-05-26,12,00:00:00
1,ممكن التواصل معي الخاص أجل تقديم شكوى ف شركة ...,"Cairo, Egypt",2022-05-23,20,00:00:00
2,ممكن التواصل الخاص تكرمتم,"Cairo, Egypt",2022-05-23,19,00:00:00
3,عايز الايميل الرسمي بتاعكم انتوا الإمارات ينف...,"Cairo, Egypt",2022-05-23,19,00:00:00
4,اتصالات مصر معندهاش ايميل لتلقي الشكاوي وبتقو...,"Cairo, Egypt",2022-05-23,19,00:00:00
...,...,...,...,...,...
1794,ممكن تغيير بيانات الخط اقرب فرع يكون متواجد ح...,"Cairo, Egypt",2022-05-22,6,00:00:00
1795,السلام ورحمة الله وبركاته انا سافرت مصر وشريح...,"Cairo, Egypt",2022-05-21,14,00:00:00
1797,الشريحه متوقفه واريد تفعيلها مصالحنا متعطله و...,"Cairo, Egypt",2022-05-21,10,00:00:00
1813,عندك خاصية بالجوال حقك القائمة البيضاء فعلها ...,"القاهرة, مصر",2022-05-21,1,00:00:00


In [26]:
Morph = Morphological_Analysis(df, 'Tweet')


In [27]:
df = Morph.del_stopwords()
df

,Tweet,Location,date,hours,minuites
0,اتصالات شركه نصابه وبتضحك ع العملاء ومفيش اي ...,"Cairo, Egypt",2022-05-26,12,00:00:00
1,ممكن التواصل معي الخاص أجل تقديم شكوى ف شركة ...,"Cairo, Egypt",2022-05-23,20,00:00:00
2,ممكن التواصل الخاص تكرمتم,"Cairo, Egypt",2022-05-23,19,00:00:00
3,عايز الايميل الرسمي بتاعكم انتوا الإمارات ينف...,"Cairo, Egypt",2022-05-23,19,00:00:00
4,اتصالات مصر معندهاش ايميل لتلقي الشكاوي وبتقو...,"Cairo, Egypt",2022-05-23,19,00:00:00
...,...,...,...,...,...
1794,ممكن تغيير بيانات الخط اقرب فرع يكون متواجد ح...,"Cairo, Egypt",2022-05-22,6,00:00:00
1795,السلام ورحمة الله وبركاته انا سافرت مصر وشريح...,"Cairo, Egypt",2022-05-21,14,00:00:00
1797,الشريحه متوقفه واريد تفعيلها مصالحنا متعطله و...,"Cairo, Egypt",2022-05-21,10,00:00:00
1813,عندك خاصية بالجوال حقك القائمة البيضاء فعلها ...,"القاهرة, مصر",2022-05-21,1,00:00:00


In [28]:
df.fillna(value=" ", inplace=True)
df

,Tweet,Location,date,hours,minuites
0,اتصالات شركه نصابه وبتضحك ع العملاء ومفيش اي ...,"Cairo, Egypt",2022-05-26,12,00:00:00
1,ممكن التواصل معي الخاص أجل تقديم شكوى ف شركة ...,"Cairo, Egypt",2022-05-23,20,00:00:00
2,ممكن التواصل الخاص تكرمتم,"Cairo, Egypt",2022-05-23,19,00:00:00
3,عايز الايميل الرسمي بتاعكم انتوا الإمارات ينف...,"Cairo, Egypt",2022-05-23,19,00:00:00
4,اتصالات مصر معندهاش ايميل لتلقي الشكاوي وبتقو...,"Cairo, Egypt",2022-05-23,19,00:00:00
...,...,...,...,...,...
1794,ممكن تغيير بيانات الخط اقرب فرع يكون متواجد ح...,"Cairo, Egypt",2022-05-22,6,00:00:00
1795,السلام ورحمة الله وبركاته انا سافرت مصر وشريح...,"Cairo, Egypt",2022-05-21,14,00:00:00
1797,الشريحه متوقفه واريد تفعيلها مصالحنا متعطله و...,"Cairo, Egypt",2022-05-21,10,00:00:00
1813,عندك خاصية بالجوال حقك القائمة البيضاء فعلها ...,"القاهرة, مصر",2022-05-21,1,00:00:00


In [29]:
df.to_csv('last_n_of_samples.csv', index=False)

Get Words' Root

In [ ]:
#Morph.get_lex(df, 'Tweet')
df.fillna(value=" ", inplace=True)
#df

Get Sentiments

In [ ]:
Sent = Sentiment_Analysis
Sent.get_sentiment(df, 'Tweet')

In [ ]:
df.to_csv('last_n_of_samples.csv', index=False)

In [ ]:
df = pd.read_csv('last_n_of_samples.csv')
df

In [ ]:
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tagger.default import DefaultTagger

mled = MLEDisambiguator.pretrained()
tagger = DefaultTagger(mled, 'pos')

#tagger.tag('و عملية برقم تم استرداد المبلغ الي محفظة فودا'.split())

df = pd.read_csv('Tweets.csv')

mapping = {}

def Replace_Row(rows):
    Cleaned_Sent = ' '
    Split_Sent = rows.split()

    for words in Split_Sent:

        if words in mapping:
            Tags = mapping[words]
        else:
            Tags = tagger.tag(words.split())
            mapping[words] = Tags

        if Tags[0] != 'conj' and Tags[0] != 'prep':
            Cleaned_Sent = Cleaned_Sent + words + " "
    return Cleaned_Sent

df['Tweet'] = df['Tweet'].apply(Replace_Row)




    


In [ ]:
df = df.drop_duplicates(subset = 'Tweet')

In [ ]:
df